# Sankey diagram for switching communities
- [Main source code reference](https://github.com/bhattbhavesh91/sankey-diagram-plotly-tutorial/blob/main/sankey-diagram-notebook.ipynb
)
- [Plotly rendering document](https://plotly.com/python/renderers/)
- Also contains code for creating custom LUT files for FSLEyes
    - Move this code to it's own function for OAM project.

Define the source and destination lists.

In [1]:
import Towlson_group_code.data_io as myFunc
import OVARIAN_functions as ov_fct
from collections import Counter
import pandas as pd
# Import the sankey function from the sankey module within pySankey
import plotly.graph_objects as go
import plotly.io as pio
pio.renderers.default = "browser"

colors = {
    "EF DMN": "#FF3333",
    "EF 2 DMN": "#FF3333",
    "LF DMN": "#FF3333",
    "ML DMN": "#FF3333",
    "EF Somamotor": "#66B2FF",
    "EF 2 Somamotor": "#66B2FF",
    "LF Somamotor": "#66B2FF",
    "ML Somamotor": "#66B2FF",
    "EF Visual": "#B266FF",
    "EF 2 Visual": "#B266FF",
    "LF Visual": "#B266FF",
    "ML Visual": "#B266FF",
    "EF Frontoparietal": "#FFB266",
    "EF 2 Frontoparietal": "#FFB266",
    "LF Frontoparietal": "#FFB266",
    "ML Frontoparietal": "#FFB266",
    "EF Dorsal Attention": "#00994C",
    "EF 2 Dorsal Attention": "#00994C",
    "LF Dorsal Attention": "#00994C",
    "ML Dorsal Attention": "#00994C",
    "EF Ventral Attention": "#FFCCFF",
    "EF 2 Ventral Attention": "#FFCCFF",
    "LF Ventral Attention": "#FFCCFF",
    "ML Ventral Attention": "#FFCCFF",
    "EF Limbic": "#F5d414",
    "EF 2 Limbic": "#F5d414",
    "LF Limbic": "#F5d414",
    "ML Limbic": "#F5d414",
    "EF Auditory": "#C90076",
    "EF 2 Auditory": "#C90076",
    "LF Auditory": "#C90076",
    "ML Auditory": "#C90076",
    "EF Unknown": "#E0E0E0",
    "EF 2 Unknown": "#E0E0E0",
    "LF Unknown": "#E0E0E0",
    "ML Unknown": "#E0E0E0"
}
# -------------------------------------------------
source = []
dest = []
# all_cycles = myFunc.load_from_pickle('../Ovarian_hormone/pickles/', 'all_cycles.pkl')
c = 'Representative Subject Level'
# cycle = all_cycles[c]
# print(cycle)
for phase in ['EF', 'LF']:
    if phase == 'EF':
        # partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/best_subject_auditory/', f'{cycle[0]}_auditory.pkl')
        partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/', 'EF_best_partition_auditory.pkl')
        # partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/', 'group_avg_20_ef_partition_sig_null_model_25092023.pkl')
    if phase == 'LF':
        # partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/hypothesis_tested/', f'cycle_{c}_LF_partition_auditory_EF.pkl')
        partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/hypothesis_tested/', 'avg_LF_partition_auditory_EF.pkl')
        # partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/', 'group_avg_20_lf_partition_ml_sig_null_model_25092023.pkl')
    # if phase == 'ML':
    #     partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/', 'group_avg_20_ml_partition_ef_sig_null_model_25092023.pkl')

    source += [f'{phase} {ov_fct.AVG_EF_FN_AUDITORY[p]}' for p in partition]

for i, phase in enumerate(['LF', 'ML']):
    if phase == 'ML':
        if i == 1:
            # partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/hypothesis_tested/', f'cycle_{c}_ML_partition_auditory_LF.pkl')
            partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/hypothesis_tested/', 'avg_ML_partition_auditory_LF.pkl')
            # partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/', 'group_avg_20_ml_partition_sig_null_model_25092023.pkl')
        if i == 0:
            # partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/hypothesis_tested/', f'cycle_{c}_ML_partition_auditory_EF.pkl')
            partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/hypothesis_tested/', 'avg_ML_partition_auditory_EF.pkl')
            # partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/', 'group_avg_20_ml_partition_ef_sig_null_model_25092023.pkl')
    if phase == 'LF':
        # partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/hypothesis_tested/', f'cycle_{c}_LF_partition_auditory_EF.pkl')
        partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/hypothesis_tested/', 'avg_LF_partition_auditory_EF.pkl')
        # partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/', 'group_avg_20_lf_partition_sig_null_model_25092023.pkl')
    # if phase == 'EF':
    #     partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/', 'group_avg_20_ef_partition_2_sig_null_model_25092023.pkl')
    #     dest += [f'{phase} 2 {ov_fct.AVG_EF_FN_AUDITORY[p]}' for p in partition]
    if phase != 'EF':
        dest += [f'{phase} {ov_fct.AVG_EF_FN_AUDITORY[p]}' for p in partition]

print(len(source))
print(len(dest))

2108
2108


Generates the values of branches to be the number of occurance of a type of "switch".

In [2]:
temp = []
for i, s in enumerate(source):
    temp.append(s + "->" + dest[i])
val_counter = Counter(temp)
data = []
ef_lf_branch_size = []
lf_ml_branch_size = []
ml_ef_branch_size = []
for s, val in val_counter.items():
    x = s.split("->")
    data.append([x[0], x[1], val])

    phase = x[0].split(" ")[0]
    source_fn = x[0].split(" ", 1)[1]
    dest_fn = x[1].split(" ", 1)[-1]
    if '2' in dest_fn:
        dest_fn = dest_fn.replace('2', ' ').strip()

    if source_fn != dest_fn:
        if phase == "EF":
            ef_lf_branch_size.append((x[0], x[1], val))
        elif phase == "LF":
            lf_ml_branch_size.append((x[0], x[1], val))
        elif phase == "ML":
            ml_ef_branch_size.append((x[0], 'EF ' + dest_fn, val))


ef_lf_sorted_branches = sorted(ef_lf_branch_size, key=lambda bs: bs[2], reverse=True)
lf_ml_sorted_branches = sorted(lf_ml_branch_size, key=lambda bs: bs[2], reverse=True)
ml_ef_sorted_branches = sorted(ml_ef_branch_size, key=lambda bs: bs[2], reverse=True)

print(*ef_lf_sorted_branches, sep="\n")
print()
print(*lf_ml_sorted_branches, sep="\n")
print()
print(*ml_ef_sorted_branches, sep="\n")
print()

('EF Frontoparietal', 'LF Dorsal Attention', 29)
('EF DMN', 'LF Dorsal Attention', 21)
('EF Somamotor', 'LF Dorsal Attention', 18)
('EF Ventral Attention', 'LF Unknown', 17)
('EF Somamotor', 'LF Ventral Attention', 15)
('EF Visual', 'LF Dorsal Attention', 12)
('EF Ventral Attention', 'LF Dorsal Attention', 9)
('EF DMN', 'LF Limbic', 7)
('EF Frontoparietal', 'LF Visual', 6)
('EF Ventral Attention', 'LF Limbic', 5)
('EF Frontoparietal', 'LF Ventral Attention', 4)
('EF Frontoparietal', 'LF DMN', 4)
('EF Dorsal Attention', 'LF Ventral Attention', 3)
('EF Limbic', 'LF Dorsal Attention', 2)
('EF Frontoparietal', 'LF Limbic', 2)
('EF Somamotor', 'LF Auditory', 2)
('EF Frontoparietal', 'LF Unknown', 2)
('EF DMN', 'LF Unknown', 2)
('EF Visual', 'LF Limbic', 1)
('EF Dorsal Attention', 'LF Auditory', 1)
('EF Dorsal Attention', 'LF Somamotor', 1)
('EF Limbic', 'LF DMN', 1)

('LF Dorsal Attention', 'ML Frontoparietal', 19)
('LF Ventral Attention', 'ML Frontoparietal', 9)
('LF Dorsal Attention', 'ML

We can modify the value to show different or more information. For example, the value can be number of occurances *and*
the cycle number + PID. To do this, we would need to track the cycle number and PID of each src to destination entry above.

In [ ]:
# some code.

# Plot Sankey

In [3]:
links_df = pd.DataFrame(data, columns=['source', 'target', 'value'])
unique_source_target = list(pd.unique(links_df[['source', 'target']].values.ravel('K')))
# print(unique_source_target)
mapping_dict = {k: v for v, k in enumerate(unique_source_target)}
links_df['source'] = links_df['source'].map(mapping_dict)
links_df['target'] = links_df['target'].map(mapping_dict)
links_dict = links_df.to_dict(orient='list')
reverse_mapping_dict = {v: k for k, v in mapping_dict.items()}

fig = go.Figure(data=[go.Sankey(
    # Self-define position of node labels
    arrangement='snap',
    # Define nodes
    node = dict(
      pad = 15,
      thickness = 15,
      # line = dict(color = "black", width = 0.5),
      label =  unique_source_target,
      color =  [colors[l] for l in unique_source_target]
    ),
    # Add links
    link = dict(
      source = links_dict["source"],
      target = links_dict["target"],
      value = links_dict["value"],
      label =  links_dict["source"],
      color =  [colors[reverse_mapping_dict[s_idx]] for s_idx in links_dict["source"]]
))])
# fig.update_layout(title_text=f"Group Level Temporal Evolution of Functional Networks (20)", font_size=10)
fig.update_layout(title_text=f"Averaged Connectome: Temporal Evolution of Functional Networks (30)", font_size=10)
fig.show()

# Exporting LUT files

In [ ]:
import OVARIAN_functions as ov_fct
import importlib
importlib.reload(ov_fct)
cycle = myFunc.load_from_pickle('../Ovarian_hormone/pickles/', 'cycles_list.pkl')[18]
a = cycle[0]
a_partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/best_subject_auditory/', f'{a}_auditory.pkl')
ov_fct.create_subcortical_lut(a_partition, f'{a}')
ov_fct.create_cortical_lut(a_partition, f'{a}')

In [ ]:
a = cycle[0]
b = cycle[2]
a_partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/best_subject_auditory/', f'{a}_auditory.pkl')
b_partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/best_subject_auditory/', f'{b}_auditory.pkl')
for i in range(1054):
    if a_partition[i] != b_partition[i]:
        b_partition[i] = -1
ov_fct.create_cortical_lut(b_partition, f'{a}_{b[:2]}_rois')
ov_fct.create_subcortical_lut(b_partition, f'{a}_{b[:2]}_rois')

## Printing out switched node list

In [158]:
a = 'EF'
b = 'ML'
a_partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/', f'{a}_best_partition_auditory.pkl')
b_partition = myFunc.load_from_pickle('../Ovarian_hormone/pickles/hypothesis_tested/', f'{b}_best_partition_auditory_EF.pkl')
node_list = ov_fct.get_node_list()
switched = ""
for i in range(1054):
    if a_partition[i] != b_partition[i]:
        switched += node_list['region_name'][i] + "\n"

with open('../Ovarian_hormone/Dump/moving_nodes.txt', 'w') as f:
    f.write(switched)

